In [3]:
# 參考資料: https://opendata.taichung.gov.tw/dataset/ef43a51b-ae96-47b6-9bec-d9ee96526cb0
# 資料網址: https://datacenter.taichung.gov.tw/swagger/OpenData/a4440bd3-b97b-43c1-abce-e29e6845f8d2

import pandas

# 讀取檔案網址, 其為csv檔
df = pandas.read_csv("https://datacenter.taichung.gov.tw/swagger/OpenData/a4440bd3-b97b-43c1-abce-e29e6845f8d2")
# 先列出所有的欄位標籤, 找出自己想要的資訊
print(df.columns)
#print(df.head())
# 順便印出Record筆數
print()
print(len(df.index)) # 總數為6201, 測試時使用100筆

Index(['年', '月', '日', '時', '分', '縣市', '區', '死', '受傷', '2-30', '天候', '光線',
       '道路類別', '速限', '道路型態', '事故位置', '路面鋪裝', '路面狀態', '路面缺陷', '障礙物', '視距',
       '號誌種類', '號誌動作', '分向設施', '快車道或一般車道間', '快慢車道間', '路面邊線', '事故類型及型態', '主要肇因',
       '受傷程度', '主要傷處', '保護裝備', '行動電話', '當事者區分(類別)', '車輛用途', '當事者行動狀態',
       '駕駛資格情形', '駕駛執照種類', '飲酒情形', '車輛撞擊部位最初', '車輛撞擊部位其他', '肇事因素個別', '肇事因素主要',
       '肇事逃逸', '職業', '旅次目的', '車種', 'GPS座標(緯度)', 'GPS座標(經度)', '事故類別'],
      dtype='object')

6201


In [0]:
from google.colab import files

# 確認自己要分析的欄位
columns = ['年','月','日','時','分','縣市', '區','事故位置','事故類型及型態','主要肇因','飲酒情形','GPS座標(緯度)','GPS座標(經度)','事故類別']
df2 = df[columns]
df3 = df2.head()

df3.to_html("test.html")
files.download("test.html")

In [0]:
from math import radians, cos, sin, asin, sqrt

# def 公式計算經緯度間距離（km）
# lat1,lon1 各為靜宜大學 緯度, 經度
def geodistance(lat2,lon2):
    lat1, lon1 = 24.227268, 120.583557
    lat1, lon1, lat2, lon2 = map(radians, [float(lat1), float(lon1), float(lat2), float(lon2)]) # 經緯度轉換成弧度
    dlat = lat2-lat1
    dlon = lon2-lon1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    distance = 2*asin(sqrt(a))*6371*1000 # 地球平均半徑，6371km
    distance = round(distance/1000,3)
    return distance

In [0]:
# 建立空df4
columns2 = ['時間','縣市', '區','事故位置','事故類型及型態','主要肇因','飲酒情形','GPS座標(緯度)','GPS座標(經度)','事故類別']
df4 = pandas.DataFrame(None, columns = columns2)

# 找出距離1公里以內的車禍位置
n = 0
df2 = df[columns]
for i in range(len(df2.index)):
  if geodistance(df2.loc[i,['GPS座標(緯度)']], df2.loc[i,['GPS座標(經度)']]) <= 1:
    #print(geodistance(df2.loc[i,['GPS座標(緯度)']], df2.loc[i,['GPS座標(經度)']]))
    # 時間格式處理
    s = str(df2.iloc[i,0])+"-0"+str(df2.iloc[i,1])
      # 日
    if(df2.iloc[i,2] < 10):
      s += "-0"+str(df2.iloc[i,2])
    else:
      s += "-"+str(df2.iloc[i,2])
      # 時
    if(df2.iloc[i,3] < 10):
      s += "_0"+str(df2.iloc[i,3])
    else:
      s += "_"+str(df2.iloc[i,3])
      # 分
    if(df2.iloc[i,4] < 10):
      s += ":0"+str(df2.iloc[i,4])
    else:
      s += ":"+str(df2.iloc[i,4])
    df4.loc[n, '時間'] = s 
    df4.iloc[n,1:] = df2.iloc[i,5:]
    n += 1

# 將Missing Value填補為0
df4 = df4.fillna(value = 0)
df4.to_html("test2.html")
files.download("test2.html")

**靜宜大學, 1公里範圍內車禍地點**

In [0]:
# 使用folium視覺化工具
import folium

# 建立地圖與設定起始位置
map = folium.Map([24.227268, 120.583557], zoom_start=16)

for i in range(len(df4.index)):
  # 設定等下要出現的資訊內容
  link = folium.Html("'{}', '{}'_'{}'".format(df4.loc[i,"時間"], df4.loc[i, "縣市"], df4.loc[i,"區"] ),script = False)

  # folium.Popup, 設定等下Popup要出現的資訊
  popup = folium.Popup( link, max_width = 200 )

  # 設定Marker標記
  #print(float(df4.loc[i,['GPS座標(緯度)']]), float(df4.loc[i, ['GPS座標(經度)']]))
  mk = folium.Marker(location = [ float(df4.loc[i,['GPS座標(緯度)']]), float(df4.loc[i, ['GPS座標(經度)']]) ], 
                    icon = folium.Icon( color = 'red', icon = 'car', prefix = 'fa' ), popup = popup)
  map.add_child(mk)

map.save('map.html')

files.download('map.html')




**台中市, 3月份 統計數量**

In [0]:
df2 = df[columns]
#print(df2.head())
rank = {}

for i in range(len(df2.index)):
  if str(df2.iloc[i, 6]) not in rank.keys():
    rank[df2.iloc[i,6]] = 0
  else:
    rank[df2.iloc[i,6]] += 1

rank = sorted(rank.items(), key=lambda x: x[1], reverse = True)
#print(rank)
for i in  range(10):
  print("{}. 區域: {}, 數量: {} ".format(i+1, rank[i][0],rank[i][1]))

1. 區域: 西屯區, 數量: 980 
2. 區域: 北屯區, 數量: 695 
3. 區域: 西區, 數量: 417 
4. 區域: 南屯區, 數量: 378 
5. 區域: 大里區, 數量: 370 
6. 區域: 北區, 數量: 340 
7. 區域: 太平區, 數量: 318 
8. 區域: 東區, 數量: 289 
9. 區域: 南區, 數量: 288 
10. 區域: 烏日區, 數量: 267 
